# Normalize the data

In [6]:
import pandas as pd
from pathlib import Path

In [30]:
def rename_book(old_name):
    """Match the names between the SBLGNT and Paul version"""
    case_ = {
        "1_corinthians": "1Co",
        "galatians": "Ga",
        "philippians": "Php",
        "philomenon": "Phm",
        "romans": "Ro",
    }
    return case_.get(old_name, old_name)

In [31]:
def clean_dataset(path: str):
    """Given an encoded dataset, perform the cleaning."""
    book_title = rename_book(Path(path).stem)
    dataset = pd.read_csv(path)

    sub_dfs = []
    for row in dataset.iterrows():
        if not pd.isna(row[1]["Appearances"]):
            for app in row[1]["Appearances"].split(","):
                chapter = int(app.split(".")[0].strip())
                verse = int(app.split(".")[1].strip())
                ix = f"{book_title}:{chapter}:{verse}"
                sub_dfs.append(
                    pd.DataFrame(
                        {
                            "ix": ix,
                            "book": book_title,
                            "chapter": chapter,
                            "verse": verse,
                            "category": row[1]["Characteristics"],
                        },
                        index=[0],
                    )
                )
    return pd.concat(sub_dfs)

In [32]:
sub_dfs = []
for file in Path("../data/raw/paul_data/").glob("*.csv"):
    sub_dfs.append(clean_dataset(file))
pd.concat(sub_dfs).to_csv("../data/cleaned/top_down/top_down.csv", index=False)
